# 1. Decision Tree

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics # for accuracy calculations 

In [5]:
data = pd.read_csv('Lab 5 diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## split dataset in feature and target variable

In [35]:
feature_column = ['Pregnancies', 'Insulin', 'BMI', 'Age', 'Glucose', 'BloodPressure', 'DiabetesPedigreeFunction' ]
X = data[feature_column]
y = data.Outcome # target variable
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 13)


In [36]:
dtree =  DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

In [37]:
print('Accuracy of dtree : ', metrics.accuracy_score(y_test,y_pred))

Accuracy of dtree :  0.7327586206896551


# 2. Random Forest

In [38]:
import cudf
import numpy as np
import pandas as pd

from cuml.ensemble import RandomForestClassifer as curfc
from cuml.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier as skrfc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define Parameter
n_samples = 2**18
n_features = 399
n_info = 300
data_type = np.float32

ModuleNotFoundError: No module named 'cudf'

In [ ]:
%%time
# Generate data
X,y = make_classification(n_samples = n_samples,
                          n_features = n_features,
                         random_state = 123, 
                         n_classes  = 2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y)
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=0)



In [ ]:
%%time
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf_train = cudf.DataFrame.from_pandas(y_train.values)
y_cudf_test = cudf.DataFrame.from_pandas(y_test.values)


In [ ]:
%%time
# Scikit learn model
sk_model = skrfc(n_estimators=35, max_depth=15,
                max_features=1.0,random_state=23)
sk_model.fit(X_train, y_train)

In [ ]:
%%time
# Evaluate
sk_predict = sk_model.predict(X_test)
sk_acc = accuracy_score(y_test, sk_predict)
print('Accuracy is : ', sk_acc)


In [ ]:
%%time
cuml_model = curfc(n_estimators=35, max_depth=15,
                max_features=1.0,random_state=23)
cuml_model.fit(X_cudf_train, y_cudf_train)


In [ ]:
%%time
pred_y = cuml_model.predict(X_cudf_test)
cuml_accuracy = accuracy_score(y_cudf_test, pred_y)
print('Accuracy is : ', cuml_accuracy)

## Self Practice 

In [40]:
import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier as skrfc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define Parameter
n_samples = 2**18
n_features = 399
n_info = 300
data_type = np.float32 

In [51]:
%%time
# Generate data
X,y = make_classification(n_samples = n_samples,
                          n_features = n_features,
                         random_state = 123, 
                         n_classes  = 2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y)
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=0)

Wall time: 10.4 s


In [52]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,389,390,391,392,393,394,395,396,397,398
0,-0.790922,-2.211491,0.606373,-0.477390,-1.232549,0.969216,0.572825,-1.477323,-0.771396,-1.917872,...,-1.675288,0.106222,-0.228966,0.570611,1.480970,1.797054,1.249709,-0.414499,0.067019,0.310186
1,-1.108647,0.320253,-0.863880,1.025209,0.407875,0.376982,0.776724,1.065811,0.580613,0.062218,...,1.434583,0.787594,-0.683778,0.470666,-0.345288,-1.391274,-1.166305,0.779667,-0.719738,-1.211956
2,0.010014,0.375795,-1.009028,-0.218627,1.512119,0.853019,-1.218982,-0.451277,0.644289,-0.321081,...,1.012943,-0.045631,1.509207,-1.403679,0.522153,-1.240228,-0.488388,-2.450342,-0.410560,-1.356949
3,2.059320,-0.279521,0.589402,-1.318651,-0.052748,-0.804232,-0.237376,2.435070,0.718888,2.029876,...,0.463005,0.765487,1.323778,0.348763,0.250897,1.067381,0.840999,-0.885579,-0.964769,0.628758
4,0.606317,-1.171549,-0.526950,2.466810,0.034856,0.434716,-1.002585,0.387707,-1.027077,1.872400,...,0.605204,0.408223,0.567416,-0.073815,0.731974,1.435153,0.776052,1.204597,1.828763,1.122638


In [53]:
y.tail()

262139    1
262140    1
262141    1
262142    0
262143    1
dtype: int32

In [54]:
%%time
# Generate data
X,y = make_classification(n_samples = n_samples,
                          n_features = n_features,
                         random_state = 123, 
                         n_classes  = 2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y)
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=0)

Wall time: 9.84 s


In [ ]:
%%time
# Scikit learn model
sk_model = skrfc(n_estimators=35, max_depth=15,
                max_features=1.0,random_state=23)
sk_model.fit(X_train, y_train)

In [ ]:
%%time
# Evaluate
sk_predict = sk_model.predict(X_test)
sk_acc = accuracy_score(y_test, sk_predict)
print('Accuracy is : ', sk_acc)